## Compute Metrics

Caleb Phillips (caleb.phillips@nrel.gov) and Jenna Ruzekowicz (jenna.ruzekowicz@nrel.gov)

The purpose of this notebook is to read in computed predictions and compare them to actual observations, computing metrics. We will compute metrics and produce the summary files:

> error_metrics_summary_wtk|wtk_led_2019|wtk_led_2018.csv
  
> provider,site,height_m,model,metric,value

***FIXME: height is not carried over in model output
  
And, hour, month, sector summaries:

> results_hms_{metric}.csv
  
> provider,site,height_m,model,hour,month,sector_deg,value

In [1]:
from common import *
import pandas as pd
import numpy as np
from tqdm import tqdm
import glob
import re
from dw_tap.power_output import estimate_power_output

## ANL WTK-LED 2019, no BC, FIXME: Does this include vegetation or no?? Ask Dmitry

In [7]:
#Empty dataframes to store the final metrics into
error_dataframe_wtk_2019 = pd.DataFrame()

#Directory path of model data 
directoryPath = "03 Model Outputs/"

#For every file with model results (both wtk and wtk-led)
for file in glob.glob(directoryPath + '*.csv.bz2'):
    model_data_file = pd.read_csv(file)
    
    #If the file is a wtk_led file for 2019, we have hourly ws prediction data
    if file.find("wtk_led_2019.csv") != -1:
        #Pull the turbine id from the name of the file
        tid = re.search("_t[0-9]{3}_", file).group().replace("_", "")
        model = "lanl"
        
        #Ground truth data comes from reversing the measured power output to windspeed
        ground_truth = pd.read_csv("01 Bergey Turbine Data/prepared_and_combined.csv.bz2")
        ground_truth = ground_truth.loc[ground_truth["tid"] == tid]
        
        #Merge the two data sources (ground truth and wtk-led predicted) into single dataframe
        turbine_data = pd.DataFrame.merge(model_data_file, ground_truth, on="packet_date")
        turbine_data["error"] = turbine_data["ws-adjusted"] - turbine_data["windspeed_mps"]
        
        #Bin timeseries data by hour, month, and sector
        turbine_data['hour'] = turbine_data['packet_date'].astype('datetime64[ns]').dt.hour
        turbine_data['month'] = turbine_data['packet_date'].astype('datetime64[ns]').dt.month
        turbine_data['sector'] = sectorize(turbine_data['wd'])
        
        #Cuts out turbine ids with no ground truth data present (192 and 207 as of 4/16/23)
        if len(turbine_data["windspeed_mps"]) > 0:
            error_metrics_dict = error_metrics(turbine_data["ws-adjusted"], turbine_data["windspeed_mps"])
            error_metrics_dict["source"] = "wtk_led_2019"
            error_metrics_dict["tid"] = tid
            error_metrics_dict["model"] = model
            error_metrics_df = pd.DataFrame(error_metrics_dict, index=[0])
            error_dataframe_wtk_2019 = pd.concat([error_dataframe_wtk_2019, error_metrics_df])
            
            #Visualizations
            #Scatter and histogram plots for individual turbine ids error metrics
            #plot_scatter_and_hist(turbine_data["ws-adjusted"],turbine_data["windspeed_mps"],tid = tid, axrange=[0,16])
            
            #Polar plot for individual turbine ids error metrics
            #plotpolar(turbine_data)
            
            #12x24 heatmap for individual turbine ids error metrics
            #heat_table = plot1224heatmap(turbine_data)
            
            #Save site-specific 12x24 summaries (heat_table), can only save if previous viz line is run
            #heat_table.to_csv("04 Error Metrics/1224summary_wtk_led_2019_" + tid + '_' + model + ".csv.bz2")
        

#print(error_dataframe_wtk_2019)


#Save error_dataframe_wtk_2019
error_dataframe_wtk_2019.to_csv("04 Error Metrics/error_metrics_summary_wtk_led_2019.csv.bz2")

                  datetime        ws          wd  \
0      2007-01-01 00:00:00  7.516921  221.320538   
1      2007-01-01 01:00:00  7.535241  222.903415   
2      2007-01-01 02:00:00  7.157962  223.277353   
3      2007-01-01 03:00:00  5.766696  216.183644   
4      2007-01-01 04:00:00  7.140035  229.592219   
...                    ...       ...         ...   
61363  2013-12-31 19:00:00  2.051973  205.166430   
61364  2013-12-31 20:00:00  2.524148  184.089366   
61365  2013-12-31 21:00:00  3.818210  177.280255   
61366  2013-12-31 22:00:00  3.387345  167.183998   
61367  2013-12-31 23:00:00  4.431360  154.608806   

       inversemoninobukhovlength_2m   tid                packet_date  \
0                          0.010078  t183  2007-01-01 00:00:00+00:00   
1                          0.008616  t183  2007-01-01 01:00:00+00:00   
2                          0.011590  t183  2007-01-01 02:00:00+00:00   
3                          0.020440  t183  2007-01-01 03:00:00+00:00   
4              

KeyboardInterrupt: 

In [ ]:
error_dataframe_wtk = pd.DataFrame()
#Directory path of model data 
directoryPath = "03 Model Outputs/"

#For every file with model results (both wtk and wtk-led)
for file in glob.glob(directoryPath + '*.csv.bz2'):
    model_data_file = pd.read_csv(file)
    
    if file.find("wtk.csv") != -1:
        #Do processing for wtk data (<2018)
        tid = re.search("_t[0-9]{3}_", file).group().replace("_", "")
        if file.find("lanl") != -1: model = "lanl"
        elif file.find("anl") != -1: model = "anl"
        print(model_data_file)
        #TODO: Read in the hourly ws data for the turbine and convert to power production
        
        
        #TODO: Combine production data into sums for the daily values
        
        
        #Read in the ground truth data for daily summaries 
        ground_truth = pd.read_csv("01 Bergey Turbine Data/daily_summaries.csv.bz2")
        #Get rid of everything that is before 2018
        ground_truth = ground_truth[~(ground_truth['date'] >= '2017-12-31')]
        
        #TODO: Combine dataframes on date (by date)
        
        #TODO: Run error metrics for ground truth of day vs. wtk day
        
        print(ground_truth)